<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/item-based-cf/item_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
#@title Download necessary libraries
!pip install pyspark -qq
!pip install -U -q PyDrive -qq
!apt install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 122542 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headles

In [2]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive

from typing import Tuple, Callable, Dict
from functools import reduce
import pickle

In [3]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [4]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


## Setup ngrok

In [6]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=0e5ba23692511bc83de748351a5c9259dd1176c3600ae422d20225d38d6985ab
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [7]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [51]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [52]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://664a-35-221-57-162.ngrok-free.app" -> "http://127.0.0.1:4050"


In [10]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x7f3c5ba1dae0>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.name', 'PySparkTutorial'),
  ('spark.driver.port', '34167'),
  ('spa

# Data acquisition

In [11]:
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])




In [12]:
playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
# slice_df = spark.read.schema(playlist_schema).json(LITTLE_SLICE_FILE, multiLine=True)
audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

In [13]:
# slice_df.select("tracks").first()

In [14]:
# slice_df.show()

# Item-Based Collaborative Filtering

Item-Based Collaboartive Filtering is the "transpose" approach to user-based CF. This time we won't consider the users' feature vectors, but the items'.
An item's rating vector $\mathbf{r}_i$ is the vector of ratings given to the item $i$ for all the users.

Let $m$ be the number of users, $n$ the number of playlists, then $\mathbf{r}_i \in \mathbb{R}^m$ and $\mathbf{R} \in \mathbb{R}^{m \times n}$.

In order to make a prediction, we take the set of items $I_u$ rated by the user $u$, and we compute the set $I^k_u$ of the top-$k$ most similar items to $i$ rated by $u$, for each item $i \in I_u$. Once done that, we average the $k$ rating vectors weighting them by their respective similarity.

In [15]:
# slice_df = slice_df.cache()

In [16]:
DEBUG = True

In [17]:
NUM_PLAYLISTS = 100_000
SONGS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-train-{NUM_PLAYLISTS}.json")
SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-train-{NUM_PLAYLISTS}.json") #TODO: Little bug this is songs_df, meaning it hasn't got any info, but we don't actually care.
RATING_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-train-{NUM_PLAYLISTS}.txt")
with open(RATING_VECTOR_LENGTH_PATH, "r") as f:
  RATING_VECTOR_LENGTH = int(f.read()) + 1

songs_embeddings = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH)
songs_df = spark.read.json(SONGS_INFO_DF)

playlist_map_schema = StructType([
    StructField("track_uri", StringType(), True),
    StructField("embedding", VectorUDT(), True)
])
PLAYLIST_MAP_PATH = os.path.join(SAVED_DFS_PATH, f"playlist_map-{NUM_PLAYLISTS}.json")
playlist_map = spark.read.schema(playlist_map_schema).json(PLAYLIST_MAP_PATH)

In [18]:
TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")

train_df = spark.read.schema(playlist_schema).json(TRAIN_DF_PATH)
test_df = spark.read.schema(playlist_schema).json(TEST_DF_PATH)

In [19]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  """
  Computes the Jaccard Similarity between two sparse binary vectors
  """
  # Convert SparseVectors to sets
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  # Calculate the intersection and union of the sets
  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  # Calculate the similarity
  similarity = intersection / union

  return similarity

The similarity between each couple is intractable, we can cluster the similar tracks in buckets using Locally Sensitive Hashing.

In [ ]:
from pyspark.ml.feature import MinHashLSH, MinHashLSHModel

LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"lsh_model-{NUM_PLAYLISTS}.pickle")
if os.path.exists(LSH_MODEL_PATH):
  model = MinHashLSHModel.load(LSH_MODEL_PATH)
else:
  mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=5)
  model = mh.fit(playlist_map)
  model.save(LSH_MODEL_PATH)

model.transform(playlist_map).show()

Since the MinHasLSH model is not so fast, let's try with another type of LSH model

In [98]:
# from pyspark.ml.feature import BucketedRandomProjectionLSH, BucketedRandomProjectionLSHModel

# BRP_LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"brp_lsh_model-{NUM_PLAYLISTS}.pickle")
# if os.path.exists(BRP_LSH_MODEL_PATH):
#   model = BucketedRandomProjectionLSHModel.load(BRP_LSH_MODEL_PATH)
# else:
#   brp = BucketedRandomProjectionLSH(inputCol="embedding", outputCol="hashes", numHashTables=5)
#   model = brp.fit(playlist_map)
#   model.save(BRP_LSH_MODEL_PATH)

# model.transform(playlist_map).show()

+--------------------+--------------------+--------------------+
|           track_uri|           embedding|              hashes|
+--------------------+--------------------+--------------------+
|spotify:track:000...|(100001,[28824,60...|[[-1.0], [-1.0], ...|
|spotify:track:000...|(100001,[60352],[...|[[-1.0], [0.0], [...|
|spotify:track:000...|(100001,[76926],[...|[[0.0], [-1.0], [...|
|spotify:track:000...|(100001,[79686],[...|[[-1.0], [-1.0], ...|
|spotify:track:000...|(100001,[14092,21...|[[0.0], [0.0], [-...|
|spotify:track:000...|(100001,[74372],[...|[[0.0], [0.0], [-...|
|spotify:track:000...|(100001,[8758],[1...|[[0.0], [-1.0], [...|
|spotify:track:000...|(100001,[70399],[...|[[0.0], [-1.0], [...|
|spotify:track:001...|(100001,[9796,169...|[[0.0], [0.0], [-...|
|spotify:track:001...|(100001,[3698,702...|[[0.0], [0.0], [-...|
|spotify:track:001...|(100001,[92210],[...|[[0.0], [-1.0], [...|
|spotify:track:001...|(100001,[7110,250...|[[0.0], [0.0], [-...|
|spotify:track:001...|(10

In [99]:
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("distCol")).withColumnRenamed("distCol", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding", "hashes").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

if DEBUG:
  key = playlist_map.first()[1]
  neigh = model.approxNearestNeighbors(playlist_map, key, 10)
  merged_df_2 = merge_aggregate_df(transform_to_aggregate(neigh), neigh)
  merged_df_2.show() 

+--------------------+--------------------+--------------------+-----------+---+
|           track_uri|           embedding|              hashes|squared_sum|sum|
+--------------------+--------------------+--------------------+-----------+---+
|spotify:track:5YG...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:0aT...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:657...|(100001,[60767],[...|[[-1.0], [-1.0], ...|        2.0|2.0|
|spotify:track:7LU...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:6Jd...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:3eJ...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:7rU...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:2uU...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:0lm...|(100001,[28824],[...|[[0.0], [0.0], [-...|        2.0|2.0|
|spotify:track:000...|(10000

In [111]:
import time
def df_to_dict(df: DataFrame) -> DataFrame:
  return df.select("track_uri", "distCol").rdd.collectAsMap()

def merge_dicts(d1: Dict[str, float], d2: Dict[str, float]) -> Dict[str, float]:
  for key, value in d2.items():
    if key in d1:
      if type(d1[key]) is float:
        d1[key] = [d1[key]]
      d1[key] += [value]
    else:
      d1[key] = [value]
  return d1

if DEBUG:
  key = playlist_map.first()[1]
  neigh = F.broadcast(model.approxNearestNeighbors(playlist_map, key, 10)).cache()
  merged_df_2 = merge_dicts(df_to_dict(neigh), df_to_dict(neigh))
  merged_df_2

In [150]:
import time
from functools import reduce

#TODO: see if the dictionary approach is right
def extract_similar_songs(playlist_tracks: DataFrame, playlist_map, model, k=10) -> DataFrame:
  aggregate_df = None
  tracks_embedding = F.broadcast(playlist_map.join(F.broadcast(playlist_tracks), "track_uri").select("track_uri", "embedding"))
  transformed_tracks_embeddings = model.transform(tracks_embedding).cache()
  k_neighs = []

  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors'):
    k_neigh = F.broadcast(model.approxNearestNeighbors(transformed_tracks_embeddings, row["embedding"], k)).cache()
    k_neighs.append(df_to_dict(k_neigh))

  aggregate_df = reduce(merge_dicts, k_neighs)
  return aggregate_df

if DEBUG:
  # first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*").distinct()
  first_playlist = train_df.filter("pid == 1005").select(F.explode("tracks")).select("col.*").distinct()
  recommendations = extract_similar_songs(first_playlist, playlist_map, model, k=10)

Extracting k-neighbors: 100%|██████████| 21/21 [00:08<00:00,  2.52it/s]


We are safe by using python dictionaries since the data will be very small, and so dictionaries will be faster than pyspark dataframes.

In [116]:
import sys
print(f"The reccomendation dictionary is {sys.getsizeof(recommendations)} bytes")

The reccomendation dictionary is 4696 bytes


In [151]:
from typing import List
def aggregate_recommendations(recommendations: Dict[str, float | List[float]]) -> Dict[str, float]:
  aggregated = {}
  for key, value in recommendations.items():
    if type(value) is list:
      if sum(value) == 0:
        continue
      aggregated[key] = sum(x for x in value) / len(value)
    else:
      aggregated[key] = value
  return aggregated

k = 40
aggregated_recs = aggregate_recommendations(recommendations)
top_k = sorted(aggregated_recs.items(), key=lambda x: -x[1])[:k]
top_k 

[('spotify:track:1uL78kFWxhz3umTMWJu8n5', 0.8208208096471215),
 ('spotify:track:53mrVsi49rLHIaKBiSvElG', 0.8168825223744853),
 ('spotify:track:5Z3GHaZ6ec9bsiI5BenrbY', 0.8132319173145136),
 ('spotify:track:75ZvA4QfFiZvzhj2xkaWAh', 0.8070864863059125),
 ('spotify:track:2eAZfqOm4EnOF9VvN50Tyc', 0.7995183010757708),
 ('spotify:track:4knL4iPxPOZjQzTUlELGSY', 0.7946157313966445),
 ('spotify:track:6EpRaXYhGOB3fj4V2uDkMJ', 0.7732659354967721),
 ('spotify:track:5tz69p7tJuGPeMGwNTxYuV', 0.7667465890313958),
 ('spotify:track:0Fv5N0cHBsl4bzCbollCAS', 0.7109277414486239),
 ('spotify:track:0Qh38w01QRXK6KHIv0e3hb', 0.7036263013869308),
 ('spotify:track:0bXFIF7iL17TYLyx8JHziM', 0.6843219668511102),
 ('spotify:track:7F9vK8hNFMml4GtHsaXui6', 0.6755009190076482),
 ('spotify:track:7bsnTsbiwOymZWjPF9v6Di', 0.6243419279133565),
 ('spotify:track:0NiXXAI876aGImAd6rTj8w', 0.6206458806073614),
 ('spotify:track:7iDa6hUg2VgEL1o1HjmfBn', 0.4825406381697772),
 ('spotify:track:0XLOf9LhyazPX9Ld8jPiUq', 0.46811070998

Now that we have a prediction, we can put the result in a pyspark dataframe in order to be used later

In [152]:
recommendations_df = spark.createDataFrame(data=top_k, schema=["track_uri", "similarity"])
recommendations_df.show(truncate=False)

+------------------------------------+-------------------+
|track_uri                           |similarity         |
+------------------------------------+-------------------+
|spotify:track:1uL78kFWxhz3umTMWJu8n5|0.8208208096471215 |
|spotify:track:53mrVsi49rLHIaKBiSvElG|0.8168825223744853 |
|spotify:track:5Z3GHaZ6ec9bsiI5BenrbY|0.8132319173145136 |
|spotify:track:75ZvA4QfFiZvzhj2xkaWAh|0.8070864863059125 |
|spotify:track:2eAZfqOm4EnOF9VvN50Tyc|0.7995183010757708 |
|spotify:track:4knL4iPxPOZjQzTUlELGSY|0.7946157313966445 |
|spotify:track:6EpRaXYhGOB3fj4V2uDkMJ|0.7732659354967721 |
|spotify:track:5tz69p7tJuGPeMGwNTxYuV|0.7667465890313958 |
|spotify:track:0Fv5N0cHBsl4bzCbollCAS|0.7109277414486239 |
|spotify:track:0Qh38w01QRXK6KHIv0e3hb|0.7036263013869308 |
|spotify:track:0bXFIF7iL17TYLyx8JHziM|0.6843219668511102 |
|spotify:track:7F9vK8hNFMml4GtHsaXui6|0.6755009190076482 |
|spotify:track:7bsnTsbiwOymZWjPF9v6Di|0.6243419279133565 |
|spotify:track:0NiXXAI876aGImAd6rTj8w|0.6206458806073614

Putting all togheter:

In [158]:
def item_based_recommendation(playlist: DataFrame,playlist_map: DataFrame, model: MinHashLSHModel, k=50):
  playlist_songs = playlist.select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(playlist_songs, playlist_map, model, 10)
  aggregated_recommendations = aggregate_recommendations(recommendations)
  top_k = sorted(aggregated_recommendations.items(), key=lambda x: -x[1])[:k]
  recommendations_df = spark.createDataFrame(data=top_k, schema=["track_uri", "similarity"])
  return recommendations_df

if DEBUG:
  playlist = train_df.filter("pid == 2005")
  result = item_based_recommendation(playlist, playlist_map, model)
  result.show()

Extracting k-neighbors: 100%|██████████| 34/34 [00:10<00:00,  3.33it/s]


+--------------------+------------------+
|           track_uri|        similarity|
+--------------------+------------------+
|spotify:track:1al...|0.8633717628185147|
|spotify:track:4zz...|0.8122753639146489|
|spotify:track:1wM...|0.8081843971271243|
|spotify:track:3Gp...| 0.796220408044011|
|spotify:track:3lB...|0.7923481512315753|
|spotify:track:7lG...|0.7813832653677002|
|spotify:track:7vR...|0.7724312512781818|
|spotify:track:7IW...|0.7538107392852691|
|spotify:track:39a...|0.7518138665443204|
|spotify:track:0fi...|0.7437276588861723|
|spotify:track:27v...| 0.738742752960684|
|spotify:track:6mj...|0.7263680068200388|
|spotify:track:0zM...|0.7133843830389912|
|spotify:track:1fn...|0.7133823834089543|
|spotify:track:0QL...| 0.712460732490682|
|spotify:track:1Np...|0.7052846392233619|
|spotify:track:11b...|0.6620996960181076|
|spotify:track:6NO...|0.6563514190493702|
|spotify:track:1BP...|0.6453017871192537|
|spotify:track:7Gg...|0.6162480252764613|
+--------------------+------------

# Performance Evaluation

In [134]:
def precision_at_k(recommendations, ground_truth, num_of_recommendations) -> float:
    """
    Calculates precision at k for the recommendations.
    """
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").cache()
    reccomended_relevant_tracks_count = recommended_relevant_tracks.count() #this can be top_n_results.join in order to be more performant
    recommended_relevant_tracks.unpersist()
    precision = reccomended_relevant_tracks_count / float(num_of_recommendations)

    return precision


import math
#TODO: make it more efficient somehow
def normalized_discounted_cumulative_gain(recommendations: DataFrame, ground_truth: DataFrame, num_of_recommendations: int) -> float:
  recommendations = recommendations.orderBy(F.col("similarity").desc())
  recommendations_list = recommendations.collect()
  cumulative_gain = 0

  intersection = recommendations.join(ground_truth, "track_uri").count()
  if intersection == 0: return 0

  ideal_cumulative_gain = 1 + sum((1 / math.log(i, 2)) for i in range(2, 2+intersection))
  for index, row in enumerate(recommendations_list):
    i = index + 1
    is_rel = ground_truth.filter(F.col("track_uri").isin(row.track_uri)).count() > 0
    rel = 1 if is_rel else 0
    if i == 1:
      cumulative_gain += rel
    else:
      cumulative_gain += (rel / math.log(i, 2))
  return cumulative_gain / ideal_cumulative_gain

Executing performance evaluation on a random sample of the test set

In [ ]:
def evaluate(pid: int) -> Tuple[DataFrame, float]:
    t1 = time.time()

    playlist_train = train_df.filter(f"pid == {pid}").cache()
    playlist_test = test_df.filter(f"pid == {pid}").cache()
    ground_truth = playlist_test.select(F.explode("tracks")).select("col.*").cache()
    num_of_recommendations = ground_truth.count()

    recommendations = item_based_recommendation(playlist_train, playlist_map, model, k=num_of_recommendations)

    precision = precision_at_k(recommendations, ground_truth, num_of_recommendations)
    gain = normalized_discounted_cumulative_gain(recommendations, ground_truth, num_of_recommendations)

    t2 = time.time()
    print(f"Total time: {t2-t1}")

    playlist_train.unpersist()
    playlist_test.unpersist()
    ground_truth.unpersist()

    return playlist_train, playlist_test, ground_truth, recommendations, precision, gain

if DEBUG:
  train, test, gt, rec, prec, gain  = evaluate(1003)
  train.show(), test.show(), gt.show(), rec.show(truncate=False)
  print(f"Precision: {prec}, Gain: {gain}")

Extracting k-neighbors:  52%|█████▏    | 87/168 [00:10<00:18,  4.45it/s]

In [130]:
import json
EVALUATION_RESULTS_PATH = os.path.join(GDRIVE_DATA_DIR, "item_based_evaluation_results")
def perform_evaluation():
  SAMPLING_FRACTION = 0.01
  sampled_playlists = train_df.sample(False, SAMPLING_FRACTION, seed=42).cache()
  results = []
  for row in tqdm(sampled_playlists.collect(), desc="Performing evaluation"):
      pid = row['pid']
      train, test, gt, rec, prec, gain = evaluate(pid)
      results.append((prec, gain))
  with open(EVALUATION_RESULTS_PATH, "w") as f:
    json.dump(results, f)
  return results

results = perform_evaluation()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

# All against all tries

Since computing the k nearest neighbors is super slow, I can pre-compute them offline and store them. This will require like 100 years lol.

In [106]:
mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=5)
model = mh.fit(playlist_map)

In [108]:
def compute_all_k_neighbors(playlist_map: DataFrame, model) -> DataFrame:
    result = []
    transformed_playlist_map = model.transform(playlist_map).cache()
    for index, row in enumerate(tqdm(playlist_map.collect(), desc="Computing k-neighbors")):
        k_neighs = model.approxNearestNeighbors(transformed_playlist_map, row.embedding, 10).select("track_uri", F.col("distCol").alias("similarity"))
        result.append((row.track_uri, k_neighs.collect()))

    k_neighs_schema = StructType([
        StructField("track_uri", StringType(), nullable=True),
        StructField("distCol", FloatType(), nullable=True)
    ])

    schema = StructType([
        StructField("track_uri", StringType(), nullable=True),
        StructField("k_neighs", ArrayType(k_neighs_schema), nullable=True)
    ])

    result_df = spark.createDataFrame(result, schema)
    transformed_playlist_map.unpersist()

    return result_df

result_df = compute_all_k_neighbors(playlist_map, model)

Computing k-neighbors:   0%|          | 176/681805 [01:27<71:46:32,  2.64it/s]ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
Computing k-neighbors:   0%|          | 176/681805 [01:27<94:38:14,  2.00it/s]


KeyboardInterrupt: ignored

In [ ]:
K_NEIGHBOURS_PATH = os.path.join(GDRIVE_DATA_DIR,"saved_models", f"k_neighbours-{NUM_PLAYLISTS}.parquet")
result_df.write.parquet(K_NEIGHBOURS_PATH)

# Old stuff

In [ ]:
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("similarity")).withColumnRenamed("similarity", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

In [ ]:
def create_similarity_df(input_vector: SparseVector, playlist_map: DataFrame, similarity_function: Callable) -> DataFrame:
  """
  Returns the similarity df for a single song
  """
  
  @F.udf(returnType=FloatType())
  def compute_similarity(vector1):
    return similarity_function(vector1, input_vector)

  result_df = playlist_map.withColumn("similarity", compute_similarity(playlist_map.embedding))
  
  return result_df

def get_top_k_results(track_uri: str, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  return similarity_df.filter((F.col("similarity") > 0) & (F.col("track_uri") != track_uri)).orderBy(F.col("similarity").desc()).limit(k)

if DEBUG:
  first_song_vector = playlist_map.select("embedding").limit(1).first()[0]
  first_track_uri = playlist_map.select("track_uri").limit(1).first()[0]
  similarity_df = create_similarity_df(first_song_vector, playlist_map, jaccard_similarity)
  top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
  top_k_results.show()

In [ ]:
def extract_similar_songs(playlist_tracks: DataFrame, k=10) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_tracks.join(playlist_map, "track_uri")
  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors'):
    k_neigh = get_top_k_results(
        track_uri=row.track_uri,
        similarity_df= create_similarity_df(input_vector=row.embedding,
                                            playlist_map=playlist_map,
                                            similarity_function=jaccard_similarity),
        k = 10)
    
    k_neigh.show()
    if aggregate_df == None:
      aggregate_df = transform_to_aggregate(k_neigh).cache()
    else:
      aggregate_df = merge_aggregate_df(aggregate_df, k_neigh).cache()
    
    aggregate_df.show()
  return aggregate_df

if DEBUG:
  first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(first_playlist, model)

In [ ]:
recommendations.show()

In [ ]:
# def extract_similar_songs(playlist: DataFrame, playlist_map: DataFrame) -> Dict[str, float]:
#   """
#   For each track in the playlist, it extracts the top-k most similar results.
#   """
#   weighted_dict = {}
  
#   # F.udf(returnType=IntegerType())
#   def update_weighted_dict(track_uri: str, similarity: float):
#     nonlocal weighted_dict
#     if track_uri not in weighted_dict:
#       weighted_dict[track_uri] = (similarity, 1)
#     else:
#       curr_similarity = weighted_dict[track_uri][0]
#       count = weighted_dict[track_uri][1]
#       weighted_dict[track_uri] = (curr_similarity + similarity, count+1)
  
#   tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
#   tracks_mapped = tracks.join(playlist_map, "track_uri")

#   for track_row in tqdm(tracks_mapped.collect(), desc=f"Analyzing tracks"):
#     similarity_df = create_similarity_df(track_row.embedding, playlist_map, jaccard_similarity)
#     top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
#     top_k_results.foreach(lambda row: update_weighted_dict(row.track_uri, row.similarity))
#     top_k_results.unpersist()
  
#   return weighted_dict

# if DEBUG:
#   first_playlist = train_df.limit(1).cache()
#   weighted_dict = extract_similar_songs(first_playlist, playlist_map)

In [ ]:
def create_similarity_df(playlist: DataFrame, playlist_map: DataFrame, similarityFunction: Callable):
  # Get the current playlist's tracks
  playlist_tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
  playlist_tracks.show()
  # Let's extract the rating vector for each single track
  playlist_tracks_mapped = playlist_tracks.join(playlist_map, "track_uri")
  playlist_tracks_mapped.show()


  @F.udf(returnType=FloatType())
  def compute_similarity(input_vector: SparseVector, other_vector: SparseVector):
    return jaccard_similarity(input_vector, other_vector)

  # for row in playlist_tracks_mapped.collect():
  similarity_df = playlist_map.withColumn("similarity", compute_similarity(playlist_tracks_mapped.limit(2).embedding, playlist_map.embedding))
  # break
  
  similarity_df.show()
  return

first_playlsit = train_df.limit(1).cache()
create_similarity_df(first_playlsit, playlist_map, jaccard_similarity)

In [ ]:
first_playlsit.show()